# Creating a script to validate SQL queries

In [1]:
import pandas as pd
import json
import sqlite3
import os
import openpyxl

In [2]:
df = pd.read_excel(r"C:\Users\HP\Downloads\test queries.xlsx", sheet_name="Sheet1", header=0)

In [3]:
df.head(15)

,question,query,db_id,db_group,annotator
0,show number of citations by publication title,"SELECT citation_num, title FROM publication GR...",academic,spider,pallavi
1,show the age of the student by city,"SELECT age, city_code FROM student GROUP BY ci...",activity_1,spider,pallavi
2,show age of the student by city,"SELECT age, city_code FROM student GROUP BY ci...",allergy_1,spider,pallavi
3,Show the number of injured caused by the ship ...,"SELECT b.injured, a.location FROM ship AS a JO...",battle_death,spider,pallavi
4,Display the bikes available in each city,"SELECT b.bikes_available, a.city FROM station ...",bike_1,spider,pallavi
5,show weight of people by birth place,"SELECT weight, birth_place FROM people GROUP B...",body_builder,spider,pallavi
6,Display the price of a book by its title,"SELECT b.price, a.title FROM book AS a JOIN pu...",book_2,spider,pallavi
7,show the Weight of people by height,"SELECT weight, height FROM people BY height",candidate_poll,spider,pallavi
8,show unit price of invoice line by quantity,"SELECT unitprice, quantity FROM invoiceline GR...",chinook_1,spider,pallavi
9,show capacity of cinema by each location,"SELECT capacity, location FROM cinema GROUP BY...",cinema,spider,pallavi


In [4]:
df = df[['question', 'query','db_id','annotator']]
df = df.dropna(axis=0)

In [5]:
df = df.rename(columns={'annotator': 'data_source'})
df['data_source'] = df['data_source'].apply(lambda x: 'contractor_' + str(x))
#df['db_id'] = df['db_id'].apply(lambda x: x.replace('_oac_display', '')

In [6]:
df.isnull().sum()

question       0
query          0
db_id          0
data_source    0
dtype: int64

In [7]:
dict_list = []
for i, row in df.iterrows():
    row_dict = {}
    for col in df.columns:
        row_dict[col] = row[col]
    dict_list.append(row_dict)

In [8]:
with open('test queries.json', 'w') as f:
    # write the list of dictionaries to the file
    json.dump(dict_list, f, indent=4)

In [9]:
# importing JSON file 
json_file_path = r"C:\Users\HP\test queries.json" 
# importing SQLite files
sqlite_files_path = r"C:\Users\HP\Desktop\spider2"

# Loading the JSON file
with open(json_file_path, "r") as f:
    data = json.load(f)

# Iterate through each object in the JSON data
for obj in data:
    db_id = obj["db_id"]
    query = obj["query"]
    sqlite_file_name = db_id + ".sqlite"
    sqlite_file_path = os.path.join(sqlite_files_path, sqlite_file_name)
    # Check if the SQLite file exists
    if os.path.isfile(sqlite_file_path):
        # Try to parse the SQL query
        try:
            conn = sqlite3.connect(sqlite_file_path)
            cursor = conn.cursor()
            cursor.execute(query)
            conn.close()
            obj["error"] = None # No error, set to None
        except sqlite3.Error as e:
            obj["error"] = str(e) # Save the error message to the "error" column
            print(f"Error parsing query for {db_id}: {e}")
    else:
        obj["error"] = f"SQLite file for {db_id} not found." # Save the error message to the "error" column
        print(f"SQLite file for {db_id} not found.")

# Save the updated JSON data to a file
with open(json_file_path, "w") as f:
    json.dump(data, f, indent=4)

Error parsing query for candidate_poll: near "height": syntax error
Error parsing query for climbing: near "country": syntax error
Error parsing query for storm_record: no such column: t2.region_name
Error parsing query for allergy_1: no such column: city
Error parsing query for college_3: no such column: city
Error parsing query for dog_kennels: no such table: breed
Error parsing query for dog_kennels: no such table: breed
Error parsing query for dog_kennels: no such table: breed
Error parsing query for epinions_1: no such column: b.u_id
Error parsing query for allergy_1: no such column: city
Error parsing query for college_3: no such column: city
Error parsing query for academic: no such column: race
Error parsing query for customers_and_invoices: no such column: t2.product_price
Error parsing query for election_representative: no such column: c.mail
Error parsing query for insurance_and_eClaims: incomplete input
Error parsing query for gas_company: near "million": syntax error
Error

In [13]:
df = pd.DataFrame.from_dict(data)

# Save the DataFrame as an Excel file
excel_file_path = r"C:\Users\HP\Desktop\test queries_test.xlsx" # replace with your desired Excel file path
df.to_excel(excel_file_path, index=False)